In [25]:
import numpy as np
from matplotlib import pyplot as plt
from pyproj import Proj
from pandas import DataFrame

In [26]:
SGP_lat = 36.491
SGP_lon = -97.595

In [27]:
#pyproj object used for conversions
myProj = Proj("+proj=utm +zone=14S, +north +ellps=WGS84 +datum=WGS84 +units=m +no_defs")

In [37]:
UTMx, UTMy = myProj(SGP_lon, SGP_lat)
x, y = UTMx/1000.00, UTMy/1000.00


In [38]:
#SGP location in UTM
print x, y

625.843431953 4039.32746654


In [2]:
#Conversion Trials (not relevant to SGP)
x = (529025.00, 529114.00, 545227.00, 545582.00)
y = (7422210.00, 7422343.00, 7435702.00, 7435741.00)

In [3]:
dataframe = DataFrame(np.c_[x,y], columns = ['Meters East', 'Meters South'])
dataframe

,Meters East,Meters South
0,529025,7422210
1,529114,7422343
2,545227,7435702
3,545582,7435741


In [4]:
#myProj = Proj(proj='utm', zone='23K', ellps='WGS84', datum='WGS84', units='m')

In [13]:
lon, lat = myProj(dataframe['Meters East'].values, dataframe['Meters South'].values, inverse = True)

In [15]:
UTMx, UTMy = myProj(lon, lat)

In [16]:
bleh = DataFrame(np.c_[UTMx, UTMy, lon, lat], columns = ['UTMx', 'UTMy', 'Longitude', 'Latitude'])
bleh

,UTMx,UTMy,Longitude,Latitude
0,529025,7422210,-98.336594,66.916353
1,529114,7422343,-98.334528,66.917537
2,545227,7435702,-97.961192,67.035383
3,545582,7435741,-97.953025,67.035679
